In [8]:
import pandas as pd

from utils.general import get_data_path
from utils.get_ids import get_player_name
from utils.constants import SEASON

In [ ]:
for statistic in ["shots", "SoT", "SiB", "tpa", "npxG", "cc", "npxgi", "npg"]:
    df = pd.read_csv(f'{statistic}.csv')
    for gw in range(1, 35):
        print(f'GW{gw} {statistic} {df[df['gw'] == gw].shape}')

GW1 shots (780, 3)
GW2 shots (780, 3)
GW3 shots (780, 3)
GW4 shots (780, 3)
GW5 shots (780, 3)
GW6 shots (780, 3)
GW7 shots (780, 3)
GW8 shots (780, 3)
GW9 shots (780, 3)
GW10 shots (780, 3)
GW11 shots (780, 3)
GW12 shots (780, 3)
GW13 shots (780, 3)
GW14 shots (780, 3)
GW15 shots (780, 3)
GW16 shots (780, 3)
GW17 shots (780, 3)
GW18 shots (780, 3)
GW19 shots (780, 3)
GW20 shots (780, 3)
GW21 shots (780, 3)
GW22 shots (780, 3)
GW23 shots (780, 3)
GW24 shots (780, 3)
GW25 shots (780, 3)
GW26 shots (780, 3)
GW27 shots (780, 3)
GW28 shots (780, 3)
GW29 shots (780, 3)
GW30 shots (780, 3)
GW31 shots (780, 3)
GW32 shots (780, 3)
GW33 shots (780, 3)
GW34 shots (780, 3)
GW1 SoT (780, 3)
GW2 SoT (780, 3)
GW3 SoT (780, 3)
GW4 SoT (780, 3)
GW5 SoT (780, 3)
GW6 SoT (780, 3)
GW7 SoT (780, 3)
GW8 SoT (780, 3)
GW9 SoT (780, 3)
GW10 SoT (780, 3)
GW11 SoT (780, 3)
GW12 SoT (780, 3)
GW13 SoT (780, 3)
GW14 SoT (780, 3)
GW15 SoT (780, 3)
GW16 SoT (780, 3)
GW17 SoT (780, 3)
GW18 SoT (780, 3)
GW19 SoT (780,

In [26]:
for gw in range(1, 35):
    df = pd.read_csv(get_data_path('2024-25', f'gws/gw{gw}.csv'))
    print(df.shape)

(616, 17)
(627, 17)
(648, 17)
(659, 17)
(661, 17)
(664, 17)
(666, 17)
(667, 17)
(670, 17)
(674, 17)
(678, 17)
(684, 17)
(690, 17)
(693, 17)
(630, 17)
(701, 17)
(701, 17)
(705, 17)
(709, 17)
(711, 17)
(724, 17)
(729, 17)
(756, 17)
(831, 17)
(858, 17)
(783, 17)
(784, 17)
(789, 17)
(639, 17)
(791, 17)
(792, 17)
(869, 17)
(954, 17)
(645, 17)


In [33]:
for statistic in ["shots", "SoT", "SiB", "tpa", "npxG", "cc", "npxgi", "npg"]:
    df = pd.read_csv(f'{statistic}_ids.csv')
    df.fillna(0, inplace=True)
    df.to_csv(f'proccessed_{statistic}_ids.csv', index=False)

In [34]:
gw_df = pd.read_csv(get_data_path('2024-25', 'gws/gw1.csv'))
gw_df.shape

stat_df = pd.read_csv('proccessed_shots_ids.csv')
gw1_stat_df = stat_df[stat_df.gw == 1]

In [38]:
merged_df = pd.merge(gw_df, gw1_stat_df[['player_id', 'shots']], on='player_id', how='left')
null_rows = merged_df[merged_df.isnull().any(axis=1)]
null_rows['web_name'] = null_rows['player_id'].apply(get_player_name, args=(SEASON,))
null_rows

C:\Users\kayya\AppData\Local\Temp\ipykernel_15012\116146181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_rows['web_name'] = null_rows['player_id'].apply(get_player_name, args=(SEASON,))


,player_id,minutes,in_starting_xi,was_home,goals_scored,assists,goals_conceded,clean_sheets,xG,xA,xGC,xGI,total_points,gameweek,oppeonent_team_id,fixture,value,shots,web_name
169,165,0,0,True,0,0,0,0,0.00,0.00,0.00,0.00,0,1,13,9,50,NaN,Deivid
218,213,0,0,False,0,0,0,0,0.00,0.00,0.00,0.00,0,1,4,8,45,NaN,Rak-Sakyi
232,225,5,0,True,0,0,1,0,0.00,0.00,0.07,0.00,1,1,5,3,40,NaN,Holgate
234,227,0,0,True,0,0,0,0,0.00,0.00,0.00,0.00,0,1,5,3,40,NaN,J.Virginia
236,229,0,0,True,0,0,0,0,0.00,0.00,0.00,0.00,0,1,5,3,50,NaN,Maupay
243,235,90,1,True,0,0,3,0,0.00,0.02,1.43,0.02,1,1,5,3,50,NaN,Pickford
246,238,65,1,True,0,0,3,0,0.00,0.17,1.23,0.17,-2,1,5,3,45,NaN,Young
250,242,0,0,False,0,0,0,0,0.00,0.00,0.00,0.00,0,1,14,1,40,NaN,Benda
254,246,0,0,False,0,0,0,0,0.00,0.00,0.00,0.00,0,1,14,1,45,NaN,Harris
256,248,90,1,False,0,0,1,0,0.00,0.00,2.43,0.00,3,1,14,1,50,NaN,Leno
